# <img width=400 src="http://www.numpy.org/_static/numpy_logo.png" alt="Numpy"/>

- Library for numerical computations in python
- Main features:
    - fast, strongly typed, homogeneous arrays
    - A lot of highly optimised code to operate on those arrays
    - Elementwise operations
    - Slicing
    - Masks

In [ ]:
import numpy as np

In [ ]:
# Create an array from a python list
a = np.array([1, 2, 3, 4, 5])

### Some useful properties

In [ ]:
len(a)

In [ ]:
a.shape

In [ ]:
a.dtype

In [ ]:
a.ndim

In [ ]:
a.size

## Basic math

In [ ]:
2 * a

In [ ]:
a**2

In [ ]:
a**a

In [ ]:
np.cos(a)

**Attention: You need the `cos` from numpy!**

In [ ]:
import math
math.cos(a)

Most normal python functions with basic operators like `*`, `+`, `**` simply work because
of operator overloading:

In [ ]:
def poly(x):
    return x + 2 * x**2 - x**3

poly(a)

In [ ]:
poly(np.pi)

## Arbitrary dimension arrays

In [ ]:
# two-dimensional array
y = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

y + y

In [ ]:
## since python 3.5 @ is matrix product
y @ y

## Helpers for creating arrays

In [ ]:
np.zeros(10)

In [ ]:
np.ones((5, 2))

In [ ]:
np.full(5, np.nan)

In [ ]:
np.empty(5)  # attention, uninitialised memory, be carefull

In [ ]:
np.linspace(0, 1, 11)

In [ ]:
# like range() for arrays:
np.arange(0, 10)

In [ ]:
np.logspace(-4, 5, 10)

## Numpy Indexing

* Element access
* Slicing

In [ ]:
x = np.arange(0, 10)

# like lists:
x[4]

In [ ]:
# all elements with indices ≥1 and <4:
x[1:4]

In [ ]:
# negative indices count from the end
x[-1], x[-2]

In [ ]:
# combination:
x[3:-2]

In [ ]:
# step size
x[::2]

In [ ]:
# trick for reversal: negative step
x[::-1]

In [ ]:
y = np.array([x, x + 10, x + 20, x + 30])
y

In [ ]:
# comma between indices
y[3, 2:-1]

In [ ]:
# only one index ⇒ one-dimensional array
y[2]

In [ ]:
# other axis: (: alone means the whole axis)
y[:, 3]

In [ ]:
# inspecting the number of elements per axis:
y.shape

# Changing array content

In [ ]:
y

In [ ]:
y[:, 3] = 0
y

Using slices on both sides

In [ ]:
y[:,0] = x[3:7]
y

Transposing inverts the order of the dimensions

In [ ]:
y

In [ ]:
y.shape

In [ ]:
y.T

In [ ]:
y.T.shape

# Masks

* A boolean array can be used to select only the element where it contains `True`.
* Very powerfull tool to select certain elements that fullfill a certain condition

In [ ]:
a = np.linspace(0, 2, 11)
b = np.random.normal(0, 1, 11)

print(b >= 0)
print(a[b >= 0])

In [ ]:
a[b < 0] = 0
a

## Reduction operations

Numpy has many operations, which reduce dimensionality of arrays

In [ ]:
x = np.random.normal(0, 1, 10)

In [ ]:
np.sum(x)

In [ ]:
np.prod(x)

In [ ]:
np.mean(x)

Standard Deviation

In [ ]:
np.std(x)

Standard error of the mean

In [ ]:
np.std(x, ddof=1) / np.sqrt(len(x))

Sample Standard Deviation

In [ ]:
np.std(x, ddof=1)

Difference between neighbor elements

In [ ]:
z = np.arange(10)**2
np.diff(z)

### Reductions on multi-dimensional arrays


In [ ]:
array2d = np.arange(20).reshape(4, 5)

array2d

In [ ]:
np.sum(array2d, axis=0)

In [ ]:
np.mean(array2d, axis=1)

### Random numbers

* numpy has a larger number of distributions builtin

In [ ]:
np.random.uniform(0, 1, 5)

In [ ]:
np.random.normal(5, 10, 5)

## Structured numpy arrays

## Linear Algebra